In [412]:
import pandas as pd
from datetime import date
import re

In [42]:
df = pd.read_csv('data/glassdoor_jobs.csv')

## explore the data 

In [43]:
df.shape

(956, 15)

In [44]:
df.columns

Index(['Unnamed: 0', 'Job Title', 'Salary Estimate', 'Job Description',
       'Rating', 'Company Name', 'Location', 'Headquarters', 'Size', 'Founded',
       'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors'],
      dtype='object')

In [45]:
# count missing value occurrences
df[(df == '-1') | (df == -1)].count()

Unnamed: 0             0
Job Title              0
Salary Estimate      214
Job Description        0
Rating                34
Company Name           0
Location               0
Headquarters          11
Size                  11
Founded               97
Type of ownership     11
Industry              39
Sector                39
Revenue               11
Competitors          634
dtype: int64

In [46]:
df.select_dtypes(include=['number']).head()

,Unnamed: 0,Rating,Founded
0,0,3.8,1973
1,1,3.4,1984
2,2,4.8,2010
3,3,3.8,1965
4,4,2.9,1998


In [47]:
df.select_dtypes(include=['object']).head()

,Job Title,Salary Estimate,Job Description,Company Name,Location,Headquarters,Size,Type of ownership,Industry,Sector,Revenue,Competitors
0,Data Scientist,$53K-$91K (Glassdoor est.),"Data Scientist\nLocation: Albuquerque, NM\nEdu...",Tecolote Research\n3.8,"Albuquerque, NM","Goleta, CA",501 to 1000 employees,Company - Private,Aerospace & Defense,Aerospace & Defense,$50 to $100 million (USD),-1
1,Healthcare Data Scientist,$63K-$112K (Glassdoor est.),What You Will Do:\n\nI. General Summary\n\nThe...,University of Maryland Medical System\n3.4,"Linthicum, MD","Baltimore, MD",10000+ employees,Other Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),-1
2,Data Scientist,$80K-$90K (Glassdoor est.),"KnowBe4, Inc. is a high growth information sec...",KnowBe4\n4.8,"Clearwater, FL","Clearwater, FL",501 to 1000 employees,Company - Private,Security Services,Business Services,$100 to $500 million (USD),-1
3,Data Scientist,$56K-$97K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310709\n\n...,PNNL\n3.8,"Richland, WA","Richland, WA",1001 to 5000 employees,Government,Energy,"Oil, Gas, Energy & Utilities",$500 million to $1 billion (USD),"Oak Ridge National Laboratory, National Renewa..."
4,Data Scientist,$86K-$143K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee"


## to-do 
1. dealing with missing data
2. salary parsing
3. company name text only
4. state field (for both location and hq) and whether job location is in hq
5. company tenure
6. parsing of job description (python, sql, etc..) 

### dealing with missing data

In [48]:
# dropping unused columns
df = df.drop('Unnamed: 0', axis = 1)
df = df.drop('Competitors', axis = 1)

In [49]:
# filter only jobs with salary estimate
df = df[df['Salary Estimate'] != '-1']

In [50]:
df.shape

(742, 13)

### salary parsing

In [51]:
df.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue
0,Data Scientist,$53K-$91K (Glassdoor est.),"Data Scientist\nLocation: Albuquerque, NM\nEdu...",3.8,Tecolote Research\n3.8,"Albuquerque, NM","Goleta, CA",501 to 1000 employees,1973,Company - Private,Aerospace & Defense,Aerospace & Defense,$50 to $100 million (USD)
1,Healthcare Data Scientist,$63K-$112K (Glassdoor est.),What You Will Do:\n\nI. General Summary\n\nThe...,3.4,University of Maryland Medical System\n3.4,"Linthicum, MD","Baltimore, MD",10000+ employees,1984,Other Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD)
2,Data Scientist,$80K-$90K (Glassdoor est.),"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe4\n4.8,"Clearwater, FL","Clearwater, FL",501 to 1000 employees,2010,Company - Private,Security Services,Business Services,$100 to $500 million (USD)
3,Data Scientist,$56K-$97K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310709\n\n...,3.8,PNNL\n3.8,"Richland, WA","Richland, WA",1001 to 5000 employees,1965,Government,Energy,"Oil, Gas, Energy & Utilities",$500 million to $1 billion (USD)
4,Data Scientist,$86K-$143K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable


In [52]:
df['Salary Estimate'].head()

0     $53K-$91K (Glassdoor est.)
1    $63K-$112K (Glassdoor est.)
2     $80K-$90K (Glassdoor est.)
3     $56K-$97K (Glassdoor est.)
4    $86K-$143K (Glassdoor est.)
Name: Salary Estimate, dtype: object

In [53]:
# remove Glassdoor est. 
salary = df['Salary Estimate'].apply(lambda x: x.split('(')[0])

In [54]:
# remove K and $
minus_kd = salary.apply(lambda x: x.replace('K','').replace('$',''))

In [89]:
# remove per hour salary and employer provider salary benefit, then remove spacing
minus_hr = minus_kd.apply(lambda x: x.lower().replace('per hour', '').replace('employer provided salary:', '').replace(' ',''))

In [92]:
"""
max(minus_hr.apply(lambda x: len(x)))  # find out max len of the column, and filter to check max one
"""
minus_hr[minus_hr.apply(lambda x: len(x) == max(minus_hr.apply(lambda x: len(x))))]

9      120-160
10     126-201
12     106-172
15     102-190
17     118-189
18     110-175
25     109-177
39     115-180
44     110-175
46     110-150
48     150-160
49     158-211
60     120-189
61     111-176
67     107-172
79     139-220
90     100-160
94     106-172
101    117-231
110    102-164
117    200-275
124    121-193
127    120-160
128    102-163
150    112-182
154    113-223
159    118-189
161    120-145
181    124-204
182    131-207
        ...   
722    125-210
741    100-135
745    139-221
761    121-203
776    102-178
777    136-208
778    110-130
782    171-272
785    150-239
788    118-228
799    113-182
802    124-199
807    100-173
815    115-220
819    120-145
828    107-172
848    109-200
855    135-211
866    113-196
871    150-238
880    101-141
887    119-187
893    120-140
899    116-208
900    107-173
904    102-172
912    108-171
913    202-306
934    107-173
943    100-140
Name: Salary Estimate, Length: 152, dtype: object

In [96]:
# to find out any other weird data 
minus_hr.apply(lambda x: len(x))

0      5
1      6
2      5
3      5
4      6
5      6
6      5
7      6
8      5
9      7
10     7
11     6
12     7
13     5
14     6
15     7
16     6
17     7
18     7
19     6
20     6
21     6
22     6
23     6
25     7
26     6
27     6
28     5
29     6
30     5
      ..
920    5
921    6
924    6
926    6
928    6
929    5
930    6
931    5
932    5
933    5
934    7
935    5
936    5
938    6
939    5
940    5
941    6
942    5
943    7
944    6
945    6
946    6
947    6
948    6
949    6
950    6
951    6
952    5
953    6
955    6
Name: Salary Estimate, Length: 742, dtype: int64

In [98]:
df['min_salary'] = minus_hr.apply(lambda x: int(x.split('-')[0]))
df['max_salary'] = minus_hr.apply(lambda x: int(x.split('-')[1]))

In [108]:
# any missing value? 
sum(df['min_salary'].isna()) == 1 | sum(df['max_salary'].isna()) == 1

False

In [109]:
df['avg_salary'] = df['min_salary'] + df['max_salary'] / 2

In [110]:
df.columns

Index(['Job Title', 'Salary Estimate', 'Job Description', 'Rating',
       'Company Name', 'Location', 'Headquarters', 'Size', 'Founded',
       'Type of ownership', 'Industry', 'Sector', 'Revenue', 'min_salary',
       'max_salary', 'avg_salary'],
      dtype='object')

In [114]:
# pd.set_option('display.max_rows', df.shape[0]+1)
pd.set_option('display.max_rows', 10)

In [115]:
df

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,min_salary,max_salary,avg_salary
0,Data Scientist,$53K-$91K (Glassdoor est.),"Data Scientist\nLocation: Albuquerque, NM\nEdu...",3.8,Tecolote Research\n3.8,"Albuquerque, NM","Goleta, CA",501 to 1000 employees,1973,Company - Private,Aerospace & Defense,Aerospace & Defense,$50 to $100 million (USD),53,91,98.5
1,Healthcare Data Scientist,$63K-$112K (Glassdoor est.),What You Will Do:\n\nI. General Summary\n\nThe...,3.4,University of Maryland Medical System\n3.4,"Linthicum, MD","Baltimore, MD",10000+ employees,1984,Other Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),63,112,119.0
2,Data Scientist,$80K-$90K (Glassdoor est.),"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe4\n4.8,"Clearwater, FL","Clearwater, FL",501 to 1000 employees,2010,Company - Private,Security Services,Business Services,$100 to $500 million (USD),80,90,125.0
3,Data Scientist,$56K-$97K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310709\n\n...,3.8,PNNL\n3.8,"Richland, WA","Richland, WA",1001 to 5000 employees,1965,Government,Energy,"Oil, Gas, Energy & Utilities",$500 million to $1 billion (USD),56,97,104.5
4,Data Scientist,$86K-$143K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,86,143,157.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
950,"Sr Scientist, Immuno-Oncology - Oncology",$58K-$111K (Glassdoor est.),Site Name: USA - Massachusetts - Cambridge\nPo...,3.9,GSK\n3.9,"Cambridge, MA","Brentford, United Kingdom",10000+ employees,1830,Company - Public,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,$10+ billion (USD),58,111,113.5
951,Senior Data Engineer,$72K-$133K (Glassdoor est.),THE CHALLENGE\nEventbrite has a world-class da...,4.4,Eventbrite\n4.4,"Nashville, TN","San Francisco, CA",1001 to 5000 employees,2006,Company - Public,Internet,Information Technology,$100 to $500 million (USD),72,133,138.5
952,"Project Scientist - Auton Lab, Robotics Institute",$56K-$91K (Glassdoor est.),The Auton Lab at Carnegie Mellon University is...,2.6,Software Engineering Institute\n2.6,"Pittsburgh, PA","Pittsburgh, PA",501 to 1000 employees,1984,College / University,Colleges & Universities,Education,Unknown / Non-Applicable,56,91,101.5
953,Data Science Manager,$95K-$160K (Glassdoor est.),Data Science ManagerResponsibilities:\n\nOvers...,3.2,"Numeric, LLC\n3.2","Allentown, PA","Chadds Ford, PA",1 to 50 employees,-1,Company - Private,Staffing & Outsourcing,Business Services,$5 to $10 million (USD),95,160,175.0


### parsing company name

In [128]:
df['Company Name'].apply(lambda x: x[:-4])

0                          Tecolote Research
1      University of Maryland Medical System
2                                    KnowBe4
3                                       PNNL
4                         Affinity Solutions
                       ...                  
950                                      GSK
951                               Eventbrite
952           Software Engineering Institute
953                             Numeric, LLC
955             Riverside Research Institute
Name: Company Name, Length: 742, dtype: object

In [130]:
df['company_txt'] = df.apply(lambda x: x['Company Name'] if x['Rating'] < 0 else x['Company Name'][:-4], axis = 1)

In [378]:
df

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,min_salary,max_salary,avg_salary,company_txt,job_state,hq_state
0,Data Scientist,$53K-$91K (Glassdoor est.),"Data Scientist\nLocation: Albuquerque, NM\nEdu...",3.8,Tecolote Research\n3.8,"Albuquerque, NM","Goleta, CA",501 to 1000 employees,1973,Company - Private,Aerospace & Defense,Aerospace & Defense,$50 to $100 million (USD),53,91,98.5,Tecolote Research,NM,CA
1,Healthcare Data Scientist,$63K-$112K (Glassdoor est.),What You Will Do:\n\nI. General Summary\n\nThe...,3.4,University of Maryland Medical System\n3.4,"Linthicum, MD","Baltimore, MD",10000+ employees,1984,Other Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),63,112,119.0,University of Maryland Medical System,MD,MD
2,Data Scientist,$80K-$90K (Glassdoor est.),"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe4\n4.8,"Clearwater, FL","Clearwater, FL",501 to 1000 employees,2010,Company - Private,Security Services,Business Services,$100 to $500 million (USD),80,90,125.0,KnowBe4,FL,FL
3,Data Scientist,$56K-$97K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310709\n\n...,3.8,PNNL\n3.8,"Richland, WA","Richland, WA",1001 to 5000 employees,1965,Government,Energy,"Oil, Gas, Energy & Utilities",$500 million to $1 billion (USD),56,97,104.5,PNNL,WA,WA
4,Data Scientist,$86K-$143K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,86,143,157.5,Affinity Solutions,NY,NY
5,Data Scientist,$71K-$119K (Glassdoor est.),CyrusOne is seeking a talented Data Scientist ...,3.4,CyrusOne\n3.4,"Dallas, TX","Dallas, TX",201 to 500 employees,2000,Company - Public,Real Estate,Real Estate,$1 to $2 billion (USD),71,119,130.5,CyrusOne,TX,TX
6,Data Scientist,$54K-$93K (Glassdoor est.),Job Description\n\n**Please only local candida...,4.1,ClearOne Advantage\n4.1,"Baltimore, MD","Baltimore, MD",501 to 1000 employees,2008,Company - Private,Banks & Credit Unions,Finance,Unknown / Non-Applicable,54,93,100.5,ClearOne Advantage,MD,MD
7,Data Scientist,$86K-$142K (Glassdoor est.),Advanced Analytics – Lead Data Scientist\nOver...,3.8,Logic20/20\n3.8,"San Jose, CA","Seattle, WA",201 to 500 employees,2005,Company - Private,Consulting,Business Services,$25 to $50 million (USD),86,142,157.0,Logic20/20,CA,WA
8,Research Scientist,$38K-$84K (Glassdoor est.),SUMMARY\n\nThe Research Scientist I will be ta...,3.3,Rochester Regional Health\n3.3,"Rochester, NY","Rochester, NY",10000+ employees,2014,Hospital,Health Care Services & Hospitals,Health Care,$500 million to $1 billion (USD),38,84,80.0,Rochester Regional Health,NY,NY
9,Data Scientist,$120K-$160K (Glassdoor est.),isn’t your usual company. Our work is powered ...,4.6,<intent>\n4.6,"New York, NY","New York, NY",51 to 200 employees,2009,Company - Private,Internet,Information Technology,$100 to $500 million (USD),120,160,200.0,<intent>,NY,NY


### state field (for both location and hq) and whether same state and same location

due to the fact that 'headquarters' would take some time to clean up the state, and there's not too much reason to extract it, will forgo this for now, and focus on state field from location, and whether state field is the same as 'headquarters'

####  job_state

In [375]:
df['job_state'] = df['Location'].apply(lambda x: x.split(',')[1].replace(' ',''))

# most job_state has length of 2, check if theres any outlier
x = df['job_state'][df['job_state'].apply(lambda x: len(x) > 2)].tolist()

In [376]:
df.loc[df['job_state'].isin(x)][['Location', 'job_state']]

,Location,job_state
145,"Santa Fe Springs, Los Angeles, CA",LosAngeles


In [377]:
df.loc[df['job_state'].isin(x)]['Location'].tolist()[0].split(',')[2].replace(' ','')

'CA'

In [183]:
df

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,min_salary,max_salary,avg_salary,company_txt,job_state
0,Data Scientist,$53K-$91K (Glassdoor est.),"Data Scientist\nLocation: Albuquerque, NM\nEdu...",3.8,Tecolote Research\n3.8,"Albuquerque, NM","Goleta, CA",501 to 1000 employees,1973,Company - Private,Aerospace & Defense,Aerospace & Defense,$50 to $100 million (USD),53,91,98.5,Tecolote Research,NM
1,Healthcare Data Scientist,$63K-$112K (Glassdoor est.),What You Will Do:\n\nI. General Summary\n\nThe...,3.4,University of Maryland Medical System\n3.4,"Linthicum, MD","Baltimore, MD",10000+ employees,1984,Other Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),63,112,119.0,University of Maryland Medical System,MD
2,Data Scientist,$80K-$90K (Glassdoor est.),"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe4\n4.8,"Clearwater, FL","Clearwater, FL",501 to 1000 employees,2010,Company - Private,Security Services,Business Services,$100 to $500 million (USD),80,90,125.0,KnowBe4,FL
3,Data Scientist,$56K-$97K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310709\n\n...,3.8,PNNL\n3.8,"Richland, WA","Richland, WA",1001 to 5000 employees,1965,Government,Energy,"Oil, Gas, Energy & Utilities",$500 million to $1 billion (USD),56,97,104.5,PNNL,WA
4,Data Scientist,$86K-$143K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,86,143,157.5,Affinity Solutions,NY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
950,"Sr Scientist, Immuno-Oncology - Oncology",$58K-$111K (Glassdoor est.),Site Name: USA - Massachusetts - Cambridge\nPo...,3.9,GSK\n3.9,"Cambridge, MA","Brentford, United Kingdom",10000+ employees,1830,Company - Public,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,$10+ billion (USD),58,111,113.5,GSK,MA
951,Senior Data Engineer,$72K-$133K (Glassdoor est.),THE CHALLENGE\nEventbrite has a world-class da...,4.4,Eventbrite\n4.4,"Nashville, TN","San Francisco, CA",1001 to 5000 employees,2006,Company - Public,Internet,Information Technology,$100 to $500 million (USD),72,133,138.5,Eventbrite,TN
952,"Project Scientist - Auton Lab, Robotics Institute",$56K-$91K (Glassdoor est.),The Auton Lab at Carnegie Mellon University is...,2.6,Software Engineering Institute\n2.6,"Pittsburgh, PA","Pittsburgh, PA",501 to 1000 employees,1984,College / University,Colleges & Universities,Education,Unknown / Non-Applicable,56,91,101.5,Software Engineering Institute,PA
953,Data Science Manager,$95K-$160K (Glassdoor est.),Data Science ManagerResponsibilities:\n\nOvers...,3.2,"Numeric, LLC\n3.2","Allentown, PA","Chadds Ford, PA",1 to 50 employees,-1,Company - Private,Staffing & Outsourcing,Business Services,$5 to $10 million (USD),95,160,175.0,"Numeric, LLC",PA


In [226]:
# most job_state has length of 2, check if theres any outlier
x = df['job_state'][df['job_state'].apply(lambda x: len(x) > 2)].tolist()

df.loc[df['job_state'].isin(x)][['Location', 'job_state']]

In [290]:
df.loc[df['job_state'].isin(x)][['Location', 'job_state']]

,Location,job_state
145,"Santa Fe Springs, Los Angeles, CA",LosAngeles


In [289]:
df.loc[df['job_state'].isin(x)]['Location'].tolist()[0].split(',')[2].replace(' ','')

'CA'

In [291]:
# replace 'LosAngeles with CA manually'
df.loc[(145), 'job_state'] = df.loc[df['job_state'].isin(x)]['Location'].tolist()[0].split(',')[2].replace(' ','')

In [309]:
df.loc[145,['Location', 'job_state']]

Location     Santa Fe Springs, Los Angeles, CA
job_state                                   CA
Name: 145, dtype: object

In [310]:
# no more outlier
df['job_state'][df['job_state'].apply(lambda x: len(x) > 2)].tolist()

[]

In [341]:
pd.set_option('display.max_rows', 40)
print(df['job_state'].value_counts())
pd.set_option('display.max_rows', 10)

CA    152
MA    103
NY     72
VA     41
IL     40
MD     35
PA     33
TX     28
NC     21
WA     21
NJ     17
FL     16
OH     14
TN     13
DC     11
CO     11
UT     10
IN     10
WI     10
MO      9
AZ      9
AL      8
MI      6
DE      6
KY      6
GA      6
IA      5
CT      5
NE      4
LA      4
OR      4
KS      3
NM      3
MN      2
ID      2
SC      1
RI      1
Name: job_state, dtype: int64


In [342]:
df

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,min_salary,max_salary,avg_salary,company_txt,job_state
0,Data Scientist,$53K-$91K (Glassdoor est.),"Data Scientist\nLocation: Albuquerque, NM\nEdu...",3.8,Tecolote Research\n3.8,"Albuquerque, NM","Goleta, CA",501 to 1000 employees,1973,Company - Private,Aerospace & Defense,Aerospace & Defense,$50 to $100 million (USD),53,91,98.5,Tecolote Research,NM
1,Healthcare Data Scientist,$63K-$112K (Glassdoor est.),What You Will Do:\n\nI. General Summary\n\nThe...,3.4,University of Maryland Medical System\n3.4,"Linthicum, MD","Baltimore, MD",10000+ employees,1984,Other Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),63,112,119.0,University of Maryland Medical System,MD
2,Data Scientist,$80K-$90K (Glassdoor est.),"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe4\n4.8,"Clearwater, FL","Clearwater, FL",501 to 1000 employees,2010,Company - Private,Security Services,Business Services,$100 to $500 million (USD),80,90,125.0,KnowBe4,FL
3,Data Scientist,$56K-$97K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310709\n\n...,3.8,PNNL\n3.8,"Richland, WA","Richland, WA",1001 to 5000 employees,1965,Government,Energy,"Oil, Gas, Energy & Utilities",$500 million to $1 billion (USD),56,97,104.5,PNNL,WA
4,Data Scientist,$86K-$143K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,86,143,157.5,Affinity Solutions,NY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
950,"Sr Scientist, Immuno-Oncology - Oncology",$58K-$111K (Glassdoor est.),Site Name: USA - Massachusetts - Cambridge\nPo...,3.9,GSK\n3.9,"Cambridge, MA","Brentford, United Kingdom",10000+ employees,1830,Company - Public,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,$10+ billion (USD),58,111,113.5,GSK,MA
951,Senior Data Engineer,$72K-$133K (Glassdoor est.),THE CHALLENGE\nEventbrite has a world-class da...,4.4,Eventbrite\n4.4,"Nashville, TN","San Francisco, CA",1001 to 5000 employees,2006,Company - Public,Internet,Information Technology,$100 to $500 million (USD),72,133,138.5,Eventbrite,TN
952,"Project Scientist - Auton Lab, Robotics Institute",$56K-$91K (Glassdoor est.),The Auton Lab at Carnegie Mellon University is...,2.6,Software Engineering Institute\n2.6,"Pittsburgh, PA","Pittsburgh, PA",501 to 1000 employees,1984,College / University,Colleges & Universities,Education,Unknown / Non-Applicable,56,91,101.5,Software Engineering Institute,PA
953,Data Science Manager,$95K-$160K (Glassdoor est.),Data Science ManagerResponsibilities:\n\nOvers...,3.2,"Numeric, LLC\n3.2","Allentown, PA","Chadds Ford, PA",1 to 50 employees,-1,Company - Private,Staffing & Outsourcing,Business Services,$5 to $10 million (USD),95,160,175.0,"Numeric, LLC",PA


#### hq_state

In [354]:
# one record is not in hq
# [k for k in df['Headquarters'] if ',' not in k]

# df['company_txt'] = df.apply(lambda x: x['Company Name'] if x['Rating'] < 0 else x['Company Name'][:-4], axis = 1)

# df['hq_state'] = df.apply(lambda x: x['Headquarters'].split(',')[1].replace(' ','') if x['Headquarters'] != '-1' else '-1', axis = 1)

# df['hq_state'].value_counts()

['-1']

In [374]:
# df.loc[df['hq_state'].apply(lambda x: len(x) > 2),['Location', 'Headquarters', 'hq_state']]

,Location,Headquarters,hq_state
14,"Cambridge, MA","OSAKA, Japan",Japan
33,"Cambridge, MA","Basel, Switzerland",Switzerland
39,"Fremont, CA","Gottingen, Germany",Germany
57,"Armonk, NY","Zurich, Switzerland",Switzerland
59,"Albuquerque, NM","Gottingen, Germany",Germany
82,"Pittsburgh, PA","Louvain-la-Neuve, Belgium",Belgium
87,"San Jose, CA","Beijing, China",China
126,"Cambridge, MA","OSAKA, Japan",Japan
157,"New York, NY","London, United Kingdom",UnitedKingdom
185,"Boston, MA","OSAKA, Japan",Japan


#### whether location = hq

In [379]:
df.columns

Index(['Job Title', 'Salary Estimate', 'Job Description', 'Rating',
       'Company Name', 'Location', 'Headquarters', 'Size', 'Founded',
       'Type of ownership', 'Industry', 'Sector', 'Revenue', 'min_salary',
       'max_salary', 'avg_salary', 'company_txt', 'job_state', 'hq_state'],
      dtype='object')

In [382]:
df['is_loc_hq'] = df.apply(lambda x: 1 if x['Location'] == x['Headquarters'] else 0, axis = 1)

### company tenure

In [388]:
# getting current year
current_year = date.today().year

In [395]:
# getting current year
current_year = date.today().year

df['comp_age'] = df.apply(lambda x: current_year - x['Founded'] if x['Founded'] > 0 else x['Founded'], axis = 1)

In [401]:
df['comp_age'].value_counts(sort = True)

-1      50
 11     32
 13     31
 25     27
 15     24
 9      21
 10     19
 63     18
 19     18
 37     18
 14     18
 6      16
 8      15
 170    14
 146    14
 24     14
 240    14
 7      13
 56     12
 22     12
 4      12
 86     10
 21     10
 109    10
 18     10
 16     10
 108     9
 39      9
 40      9
 60      9
 26      8
 32      8
 44      8
 45      8
 53      8
 52      8
 82      8
 33      7
 172     7
 29      7
 12      6
 73      6
 17      6
 151     6
 35      6
 28      6
 55      5
 5       5
 48      5
 54      5
        ..
 106     4
 27      4
 57      4
 191     4
 30      4
 99      3
 20      3
 23      3
 78      3
 43      3
 136     3
 158     3
 31      3
 51      3
 84      3
 74      3
 49      3
 165     2
 175     2
 2       2
 34      2
 47      2
 70      2
 38      2
 46      2
 69      2
 67      2
 41      2
 59      2
 93      2
 92      1
 50      1
 76      1
 91      1
 171     1
 161     1
 209     1
 79      1
 143     1
 36      1

### job description parsing

In [416]:
df['Job Description'][0]

'Data Scientist\nLocation: Albuquerque, NM\nEducation Required: Bachelor’s degree required, preferably in math, engineering, business, or the sciences.\nSkills Required:\nBachelor’s Degree in relevant field, e.g., math, data analysis, database, computer science, Artificial Intelligence (AI); three years’ experience credit for Master’s degree; five years’ experience credit for a Ph.D\nApplicant should be proficient in the use of Power BI, Tableau, Python, MATLAB, Microsoft Word, PowerPoint, Excel, and working knowledge of MS Access, LMS, SAS, data visualization tools, and have a strong algorithmic aptitude\nExcellent verbal and written communication skills, and quantitative analytical skills are required\nApplicant must be able to work in a team environment\nU.S. citizenship and ability to obtain a DoD Secret Clearance required\nResponsibilities: The applicant will be responsible for formulating analytical solutions to complex data problems; creating data analytic models to improve data

In [429]:
lst = ['Remarkable', 'R, and others', 'red', 'honourable mention', 'r', 'weka,R,python']
string = ' '.join(lst)

In [430]:
for i in lst:
    if re.findall(pattern, i) 

'Remarkable R, and others red honourable mention r weka,R,python'

In [432]:
pattern = 'R'

re.findall